In [0]:
from collections import defaultdict

def Burnside_Table(G):
    return gap.MatTom(gap.TableOfMarks(G))

def Character_Table(G):
    return gap.Irr(G)

def partition_wrt_associated_matrix(n, myList, Matrix_func):
    """Takes a list myList of SmallGroup indices k sand returns a dictonary with keys given by return values of Matrix_func evaluated at SmallGroup(n,k)
    with values given as list of integers k of myList with the same matrices up to change of basis by a permutation matrix (i.e. perm of rows and columns)"""

    fail = gap.TransformingPermutations(gap.Irr(gap.SmallGroup(2,1)),gap.Irr(gap.SmallGroup(3,1)))

    Partition_dict = defaultdict(list)
    Partition_dict[myList[0]] = [myList[0]]

    index_already_appended = []
    for i in range(len(myList)):
        #print(f" index is {i}")
        if i in index_already_appended:
            pass
        elif i == len(myList) -1:
            #The last element of myList is not already in the dict, therefore it is unique wrt Property so it goes under its own key in the dict
            Partition_dict[myList[len(myList)-1]] = [myList[len(myList)-1]]
        else:
            Partition_dict[myList[i]] = [myList[i]] #element with index i obviously has same value under Property_func as itself.
            for j in range(i+1,len(myList)):
                G = gap.SmallGroup(n,myList[i])
                H = gap.SmallGroup(n,myList[j])
                if gap.TransformingPermutations(Matrix_func(G), Matrix_func(H)) != fail :
                    Partition_dict[myList[i]].append(myList[j])
                    index_already_appended.append(j)
    return Partition_dict

def same_char_same_burn(n_start,n_end):
    n = n_start
    while n < n_end:
        if n % 1 == 0:
            print(f"Groups of order {n_start} <= order <{n} have been checked")

        Partitions = defaultdict(list)
        k = 1
        while k > 0:
            try:
                G = gap.SmallGroup(n,k)
            except RuntimeError:
                break
            if gap.Size(gap.Centre(G)) == n: #i.e. group is abelian
                pass
            else:
                sizes_of_conj_classes = [gap.Size(c) for c in gap.ConjugacyClasses(G)]
                sizes_of_conj_classes.sort()

                derived = gap.DerivedSubgroup(G)
                F = gap.FactorGroup(G, derived)
                abelianisation = gap.IdGroup(F) if gap.Size(F) < 500 else gap.Size(F)

                C = gap.Centre(G)
                center = gap.IdGroup(C) if gap.Size(C) < 500 else gap.Size(C)

                orders_of_subgroups = [gap.Size(H) for H in gap.ConjugacyClassesSubgroups(G)]
                orders_of_subgroups.sort()

                Partitions[(tuple(sizes_of_conj_classes)), abelianisation, center, (tuple(orders_of_subgroups))].append(k)

            k+=1
        #We now have full dictionary of all groups of order n sorted by the properties above.

        #print(f"n is {n}")
        #print(f"Partitions dict is {Partitions}")
        #print("")
        for key,value in Partitions.items():
            if len(value) > 1: #i.e. there are 2 or more small groups with same properties as above.

                #print(f"Partition[key] with more than one 'k' value is {Partitions[key]}")
                #print(f"We now sort them into a new dictionary based on char table")
                #print("...")

                Same_char_table_dict = partition_wrt_associated_matrix(n,value,Character_Table)

                for c_value in Same_char_table_dict.values():
                    if len(c_value) > 1:
                        Same_burnside_table_dict = partition_wrt_associated_matrix(n,c_value,Burnside_Table)

                        #We now check if any keys in Same_burnside_table_dict have associated list with length > 1. If so we have two groups with same char and burnside table!!
                        for b_key, b_value in Same_burnside_table_dict.items():
                            if len(b_value) > 1:
                                print("**********************************************************************")
                                print(f"Two groups with the same character table  and burnside table have been found")
                                print(f"n is {n}")
                                print(f"The k values are {b_value}")
                                print("**********************************************************************")
                                print("")
        n +=1

def same_burnside_table_of_certain_groups(n,grp_index_list):
    "Takes order n and list of indices k (according to GAP library) and returns if any two groups have same Burnside table"
    Same_burnside_table_dict = partition_wrt_associated_matrix(n,grp_index_list,Burnside_Table)

    #We now check if any keys in Same_burnside_table_dict have associated list with length > 1. If so we have two groups with same burnside table!
    match_found = False
    for b_key, b_value in Same_burnside_table_dict.items():
        if len(b_value) > 1:
            print("**********************************************************************")
            print(f"Two groups with the same burnside table have been found")
            print(f"n is {n}")
            print(f"The k values are {b_value}")
            print("**********************************************************************")
            print("")
            match_found = True
    if match_found == False:
        print(f"SmallGroups({n},{grp_index_list}) checked, they have different burnside tables")

def same_char_table_of_certain_groups(n,grp_index_list):
    "Takes order n and list of indices k (according to GAP library) and returns if any two groups have same Burnside table"
    Same_char_table_dict = partition_wrt_associated_matrix(n,grp_index_list,Character_Table)

    #We now check if any keys in Same_burnside_table_dict have associated list with length > 1. If so we have two groups with same burnside table!
    match_found = False
    for c_key, c_value in Same_char_table_dict.items():
        if len(c_value) > 1:
            print("**********************************************************************")
            print(f"Two groups with the same character table have been found")
            print(f"n is {n}")
            print(f"The k values are {c_value}")
            print("**********************************************************************")
            print("")
            match_found = True
    if match_found == False:
        print(f"SmallGroups({n},{grp_index_list}) checked, they have different character tables")

#same_char_table_of_certain_groups()
#same_burnside_table_of_certain_groups()
#same_char_same_burn()